In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras import models
from keras import layers
from pynput.keyboard import Key,Controller

Using TensorFlow backend.


In [3]:
skinkernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
low_range = np.array([0, 50, 80])
upper_range = np.array([30, 200, 255])

gesture = [['Nothing', ''], ['Ok', ''], ['Peace', ''], ['Punch', ''], ['Stop', '']]

json_file = open("model.json", "r")
loaded_json_file = json_file.read()
json_file.close()

model = models.model_from_json(loaded_json_file)

model.load_weights("model.h5")

keyboard = Controller()

In [4]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
binding = open("binding.txt", "r")
i=1
for l in binding.readlines():
    line = l.strip().split(":")
    if line[1]==' ':
        gesture[i][1] = Key.space
    elif line[1]=='':
        gesture[i][1] += ''
    else:
        gesture[i][1] = gesture[i][1] + line[1]
    i = i+1
binding.close()

In [6]:
cam = cv2.VideoCapture(0)
cv2.namedWindow("test")

im_count=0

while True:
    ret, frame = cam.read()
    frame = cv2.flip(frame,1,0)
    cv2.rectangle(frame, (400, 100), (600, 300),(0,255,0),thickness=2)
    
    cv2.imshow("test", frame)
    
    if not ret:
        break
    
    roi = frame[100:300, 400:600]
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
        
    mask = cv2.inRange(hsv, low_range, upper_range)
    mask = cv2.erode(mask, skinkernel, iterations=1)
    mask = cv2.dilate(mask, skinkernel, iterations=1)
        
    mask = cv2.GaussianBlur(mask, (15,15), 1)
    
    res = cv2.bitwise_and(roi, roi, mask = mask)
    gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        
    gray = np.array(gray, dtype='float32')
    gray /= 255
        
    gray = gray.reshape((1, 200, 200, 1))
        
    a = model.predict_classes(gray)
    
    k = cv2.waitKey(1)
    
    if(gesture[a[0]][1]!=''):
        keyboard.press(gesture[a[0]][1])
        keyboard.release(gesture[a[0]][1])
        
    if k==27:
        break

cam.release()                                                               
cv2.destroyAllWindows()